In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
from pandas import DataFrame
#import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
import pathlib
from tensorflow import keras
import os
from keras.callbacks import ModelCheckpoint
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import xgboost as xgb
from sklearn import datasets
from sklearn.datasets import dump_svmlight_file
from sklearn.externals import joblib
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from keras.layers import Input, Dense    #using set model component
from keras.models import Model    #using set model 
from keras.utils import plot_model    #show model structure
from keras import layers as Layer
import keras 
from pandas import DataFrame as df
from keras.callbacks import EarlyStopping
import gzip
import pickle
from keras import backend as K
import tensorflow as tf
from keras.layers import Dense, Dropout, Input, Activation, BatchNormalization

/Users/huangeunchong/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/huangeunchong/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/huangeunchong/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/hua

In [3]:
def check_correct(predict, y):
    result = {}
    result['True-Positive'] = 0
    result['True-Negative'] = 0
    result['False-Negative'] = 0
    result['False-Positive'] = 0

    for i in range(len(predict)) :
        if predict[i] == y[i] :
            if y[i] == 0 :
                result['True-Negative'] += 1
            else :
                result['True-Positive'] += 1
        else :
            if y[i] == 0 :
                result['False-Positive'] += 1
            else :
                result['False-Negative'] += 1

    for result_k, result_v in result.items():
        print(result_k +" : "+ str(result_v))
    
    acc=(result['True-Positive']+result['True-Negative'])/len(y)
    sensitivity=result['True-Positive']/(result['True-Positive']+result['False-Negative'])
    specificity=result['True-Negative']/(result['True-Negative']+result['False-Positive'])
    print("Accuracy :", acc)
    print("Sensitivity :", sensitivity)
    print("Specificity :", specificity)
    return acc, sensitivity, specificity

In [4]:
def model_performance(x, model, y):
    hypo = model.predict_proba(x)
    pred = np.where(df(hypo)[1] > 0.5, 1, 0).flatten()
    acc, sen, spe=check_correct(pred, y)
    auc=metrics.roc_auc_score(y, df(hypo)[1])
    
    
    df_hypo=df(hypo)[1]
    df_hypo.columns=['hypothesis 1']
    
    df_pred=df(pred)
    df_pred.columns=['prediction']
    
    df_y=df(y)
    df_y.columns=['y']
    
    pred_result=pd.concat([df_y,df_hypo, df_pred],axis=1)
    
    print("Accuracy : ",acc)
    print("AUC : ",auc)
    print(" ")
    print(df(pd.crosstab(y, pred, rownames=['Actual'], colnames=['Predicted'], margins=True)))
    print(" ")
    print(df(pred_result))

    return acc ,sen, spe ,auc, pred_result

In [5]:
def model_performance_DNN(x, model, y):
    hypo = model.predict(x)
    pred = np.where(hypo > 0.5, 1, 0).flatten()
    acc, sen, spe=check_correct(pred, y)
    auc=metrics.roc_auc_score(y, hypo)
    
    df_hypo=df(hypo)
    df_hypo.columns=['hypothesis 1']
    
    df_pred=df(pred)
    df_pred.columns=['prediction']
    
    df_y=df(y)
    df_y.columns=['y']
    
    pred_result=pd.concat([df_y,df_hypo, df_pred],axis=1)
    
    print("Accuracy : ",acc)
    print("AUC : ",auc)
    print(" ")
    print(df(pd.crosstab(y, pred, rownames=['Actual'], colnames=['Predicted'], margins=True)))
    print(" ")
    print(df(pred_result))

    return acc ,sen, spe ,auc, pred_result

### Ensemble Model
##### Classification 1

In [6]:
dataframe = pd.read_csv("./IRIS_train.csv")
dataframe

,SampleID,SSPG,SSPG_classification,A1C,GLU,AG,ALB.x,ALKP,BASO,BASOAB,...,ZSWIM7,ZSWIM8,ZSWIM8.AS1,ZXDA,ZXDB,ZXDC,ZYX,ZZEF1,ZZZ3,index
0,ZJTKAE3-02,162.0,1,5.5,89,6,3.9,99,0.5,0.03,...,3.885177,5.147299,7.673803,0.000000,0.000000,0.000000,2.151853,0.000000,0.755996,1
1,ZJTKAE3-04,162.0,1,5.2,98,8,4.1,80,0.7,0.03,...,4.259629,8.062151,8.216878,2.009271,0.000000,2.009271,2.009271,0.000000,0.000000,2
2,ZJTKAE3-06,162.0,1,5.5,91,12,4.0,67,0.9,0.05,...,4.434087,5.784487,7.768989,0.000000,1.552295,0.717454,2.462038,0.717454,0.717454,3
3,ZJTKAE3-2012,162.0,1,5.1,94,5,4.2,83,1.1,0.05,...,2.076022,5.602978,7.817027,1.193186,0.000000,0.716583,2.076022,1.193186,0.716583,2
4,ZJTKAE3-2013,162.0,1,5.0,92,9,4.2,73,0.6,0.04,...,2.538052,5.708347,8.232187,1.767240,1.379998,0.000000,3.513165,0.849071,0.849071,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,ZYXQKWY-05,158.5,1,5.1,79,11,4.2,73,1.0,0.06,...,1.544607,5.070096,8.032021,2.755259,0.000000,0.000000,0.000000,0.000000,0.000000,4
339,ZYXQKWY-06,158.5,1,5.2,94,11,3.9,68,0.4,0.03,...,3.382445,6.819696,8.224836,1.747215,0.000000,0.000000,2.181309,1.123368,2.181309,3
340,ZYXQKWY-07,158.5,1,5.1,85,6,3.9,63,0.6,0.04,...,2.413523,7.612855,7.705503,1.288405,2.413523,0.000000,1.957973,1.288405,1.288405,2
341,ZYXQKWY-08,158.5,1,5.3,99,6,3.7,57,0.7,0.05,...,0.859655,5.069710,7.327889,1.394629,0.859655,0.000000,2.342845,0.000000,0.000000,4


In [7]:
value= dataframe.drop(["SampleID", "SSPG_classification", "SSPG", "index"],axis=1)
interest = dataframe.SSPG_classification

print("Information of Training Data")
print("Samples : {}".format(value.shape[0]))
print("Features : {}".format(value.shape[1]))

Information of Training Data
Samples : 343
Features : 10783


In [8]:
#MinMax
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

X = np.array(value)
min_max_scaler = MinMaxScaler()
X_MinMax = min_max_scaler.fit_transform(X)

In [9]:
train_x, test_x, train_y, test_y = train_test_split(X_MinMax, interest, test_size=0.2, random_state=3, stratify=interest)

In [10]:
rfc = RandomForestClassifier(n_estimators=350, max_depth=5, min_samples_split=4, random_state=10)
abc = AdaBoostClassifier(n_estimators=250, random_state=0, learning_rate=0.05)
gbc = GradientBoostingClassifier(n_estimators=350, learning_rate=0.05, random_state=10, min_samples_split=5)
xgb_clf = xgb.XGBClassifier(n_estimators=100, max_depth=5,learning_rate=0.05, random_state=10)

In [11]:
rfc.fit(train_x, train_y)
abc.fit(train_x, train_y)
gbc.fit(train_x, train_y)
xgb_clf.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=10,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [13]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# Random forest
y_pred_rfc = rfc.predict_proba(test_x)[:,1]
fpr_rfc, tpr_rfc, thresholds_rfc = roc_curve(test_y, y_pred_rfc)
auc_rfc = auc(fpr_rfc, tpr_rfc)

# adaboost
y_pred_abc = abc.predict_proba(test_x)[:,1]
fpr_abc, tpr_abc, thresholds_abc = roc_curve(test_y, y_pred_abc)
auc_abc = auc(fpr_abc, tpr_abc)

# Gradient Boost
y_pred_gbc = gbc.predict_proba(test_x)[:,1]
fpr_gbc, tpr_gbc, thresholds_gbc = roc_curve(test_y, y_pred_gbc)
auc_gbc = auc(fpr_gbc, tpr_gbc)

# XGboost
y_pred_xgb = xgb_clf.predict_proba(test_x)[:,1]
fpr_xgb, tpr_xgb, thresholds_xgb = roc_curve(test_y, y_pred_xgb)
auc_xgb = auc(fpr_xgb, tpr_xgb)

# Ensemble model evaluate

In [15]:
# Model predict values 
rfc_pred = rfc.predict(test_x)
abc_pred = abc.predict(test_x)
gbc_pred = gbc.predict(test_x)
xgb_pred = xgb_clf.predict(test_x)

# test_y values
actual_value_array = test_y.values

### DNN model
##### Classification 1

In [22]:
model1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10783, input_shape=(10783,),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    #tf.keras.layers.Dropout(0.5), 
    
    tf.keras.layers.Dense(8626, input_shape=(10783,),activation='relu'), 
    tf.keras.layers.BatchNormalization(axis=1, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones'),
    #tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(6469, input_shape=(8626,),activation='relu'), 
    tf.keras.layers.BatchNormalization(axis=1, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones'),
    #tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(4313, input_shape=(6469,),activation='relu'), 
    tf.keras.layers.BatchNormalization(axis=1, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones'),
    #tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(2156, input_shape=(4313,),activation='relu'), 
    tf.keras.layers.BatchNormalization(axis=1, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones'),
    #tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(1, input_shape=(2156,),activation='sigmoid'),
])

model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
hist=model1.fit(train_x, train_y, batch_size=25, epochs=100,validation_split=0.2)#,callbacks=[tensorboard]

Train on 219 samples, validate on 55 samples
Epoch 1/100
219/219 [==============================] - 114s 521ms/sample - loss: 3.5688 - acc: 0.5160 - val_loss: 132.9531 - val_acc: 0.5091
Epoch 2/100
219/219 [==============================] - 148s 678ms/sample - loss: 0.9263 - acc: 0.6575 - val_loss: 387.5581 - val_acc: 0.5091
Epoch 3/100
219/219 [==============================] - 164s 749ms/sample - loss: 0.5756 - acc: 0.7215 - val_loss: 154.8835 - val_acc: 0.5091
Epoch 4/100
219/219 [==============================] - 108s 495ms/sample - loss: 0.2963 - acc: 0.8539 - val_loss: 57.4584 - val_acc: 0.5091
Epoch 5/100
219/219 [==============================] - 121s 553ms/sample - loss: 0.1971 - acc: 0.9178 - val_loss: 31.0655 - val_acc: 0.5091
Epoch 6/100
219/219 [==============================] - 80s 367ms/sample - loss: 0.1724 - acc: 0.9224 - val_loss: 10.1729 - val_acc: 0.5091
Epoch 7/100
219/219 [==============================] - 93s 426ms/sample - loss: 0.1175 - acc: 0.9589 - val_loss: 

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import roc_curve
y_pred_keras = model1.predict(test_x).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(test_y, y_pred_keras)

from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_rfc, tpr_rfc, label='Random Forest (area = {:.3f})'.format(auc_rfc))
plt.plot(fpr_abc, tpr_abc, label='Adaboost (area = {:.3f})'.format(auc_abc))
plt.plot(fpr_gbc, tpr_gbc, label='Gradient Boost (area = {:.3f})'.format(auc_gbc))
plt.plot(fpr_xgb, tpr_xgb, label='XGBoost (area={:.3f})'.format(auc_xgb))
plt.plot(fpr_keras, tpr_keras, label='DNN (area = {:.3f})'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('IRIS validation AUC using 10,783 features')
plt.legend(loc='best')
plt.savefig('./IRIS validation AUC using 10783 features', dpi=500, bbox_inches = 'tight')

# Model evaluate

In [ ]:
# Model predict values 
DNN_pred = model1.predict(test_x)
model_performance_DNN(test_x, model1, actual_value_array)

# test set

In [ ]:
test_set = pd.read_csv("./IRIS_test.csv")
test_set

In [ ]:
test_value= test_set.drop(["SampleID", "SSPG_classification", "SSPG"],axis=1)
test_interest = test_set.SSPG_classification

print("Information of Training Data")
print("Samples : {}".format(value.shape[0]))
print("Features : {}".format(value.shape[1]))

In [ ]:
xx = np.array(test_value)
test_X_MinMax = min_max_scaler.fit_transform(xx)

In [ ]:
# Test score
randomforest_pred = rfc.predict(test_X_MinMax)
adaboost_pred = abc.predict(test_X_MinMax)
gradient_pred = gbc.predict(test_X_MinMax)
xgb_pred=xgb_clf.predict(test_X_MinMax)
# DNN model predict values
DNN_predict = model1.predict_classes(test_X_MinMax)

In [ ]:
# Random forest
y_pred_rfc2 = rfc.predict_proba(test_X_MinMax)[:,1]
fpr_rfc2, tpr_rfc2, thresholds_rfc2 = roc_curve(test_interest, y_pred_rfc2)
auc_rfc2 = auc(fpr_rfc2, tpr_rfc2)

# adaboost
y_pred_abc2 = abc.predict_proba(test_X_MinMax)[:,1]
fpr_abc2, tpr_abc2, thresholds_abc2 = roc_curve(test_interest, y_pred_abc2)
auc_abc2 = auc(fpr_abc2, tpr_abc2)

# Gradient Boost
y_pred_gbc2 = gbc.predict_proba(test_X_MinMax)[:,1]
fpr_gbc2, tpr_gbc2, thresholds_gbc2 = roc_curve(test_interest, y_pred_gbc2)
auc_gbc2 = auc(fpr_gbc2, tpr_gbc2)

# XGboost
y_pred_xgb2 = xgb_clf.predict_proba(test_X_MinMax)[:,1]
fpr_xgb2, tpr_xgb2, thresholds_xgb2 = roc_curve(test_interest, y_pred_xgb2)
auc_xgb2 = auc(fpr_xgb2, tpr_xgb2)


y_pred_keras2 = model1.predict(test_X_MinMax).ravel()
fpr_keras2, tpr_keras2, thresholds_keras2 = roc_curve(test_interest, y_pred_keras2)
auc_keras2 = auc(fpr_keras2, tpr_keras2)

plt.figure(2)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_rfc2, tpr_rfc2, label='Random Forest (area = {:.3f})'.format(auc_rfc2))
plt.plot(fpr_abc2, tpr_abc2, label='Adaboost (area = {:.3f})'.format(auc_abc2))
plt.plot(fpr_gbc2, tpr_gbc2, label='Gradient Boost (area = {:.3f})'.format(auc_gbc2))
plt.plot(fpr_xgb2, tpr_xgb2, label='XGBoost (area={:.3f})'.format(auc_xgb2))
plt.plot(fpr_keras2, tpr_keras2, label='DNN (area = {:.3f})'.format(auc_keras2))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('IRIS test AUC using 10,783 features')
plt.legend(loc='best')
plt.savefig('./IRIS test AUC using 10783 features', dpi=500, bbox_inches = 'tight')